## **RAG** : Retrieval Augumented Generation
External Data Source : Alphabet Inc.'s Annual Financial Reports

![RAG](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

![RAG](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

#### Installing Dependancies

In [ ]:
!pip install langchain-google-genai

In [ ]:
!pip install langchain_community

In [ ]:
!pip install pypdf

In [ ]:
!pip install tiktoken

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain-chroma

In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=37800bb5b07246e5da182d7f4a9056c34c877b6803d65c049fa2034d969f08ab
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.1 MB/s eta 0:00:00


In [ ]:
!pip install pi-heif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.3 MB/s eta 0:00:00


In [ ]:
!pip install "unstructured[local-inference]"

#### Document Loading

In [ ]:
from unstructured.partition.pdf import partition_pdf
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("goog-10-k-2024.pdf")
doc = loader.load()

#### Document Chunking

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(doc)

#### Derfining Embeddings Model

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "<api-key>"

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

#### Inserting Embeddings in Vector Store

In [ ]:
from langchain_chroma import Chroma

vector_db = Chroma.from_documents(chunks, embeddings_model)

In [ ]:
retriever = vector_db.as_retriever()

#### Defining the RAG Pipeline

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

#### Checking with relevant questions

In [ ]:
chain.invoke("What was alphabet's Total revenues")

"Based on the provided text, Alphabet's **Consolidated revenues** for the year ended December 31, 2024 was **$350,018** (millions)."

In [ ]:
chain.invoke("What was alphabet's Total revenues in APAC")

"Based on the provided text, Alphabet's total revenues in APAC for the year ended December 31, 2024, was $56,815 million."

#### Checking with Irrelevant questions

In [ ]:
chain.invoke("What is text embedding and how does langchain help in doing it")

'The provided text does not mention "text embedding" or "Langchain." Therefore, I cannot answer your question.'